In [38]:
!pip3 install pandas
!pip3 install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


REMIND!!! database can't be shared. To get database.csv go to your cloud :)

In [39]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
from collections import Counter
import re

import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

database = pd.read_csv('database.csv')


In [40]:
# преобразование даты и времени в сколько прошло минут с полуночи
database['TIMESTAMP'] = pd.to_datetime(database['TIMESTAMP'])
database['MINUTES'] = database['TIMESTAMP'].dt.hour * 60 + database['TIMESTAMP'].dt.minute
database.drop(columns=['TIMESTAMP'], inplace=True)

# удалени дубликатов с заниженым прайсом
database.sort_values(by=['SERVICE', 'ORDER_PRICE'], ascending=[True, False], inplace=True)
database.drop_duplicates(subset='SERVICE', keep='first', inplace=True)
database.drop(columns=['SERVICE'], inplace=True)

# преобразование платформы в число от 0 - 2
def replace_platforms(platforms_str):
    platform_list = platforms_str.strip().split(' ')
    platform_dict = {'PC': 0, 'XBOX': 1, 'PS4': 2, 'platform?': 0}
    replaced_list = [platform_dict[platform] for platform in platform_list]
    return replaced_list[0]

database['PLATFORM'] = database['CHANNEL'].apply(replace_platforms)
database.drop(columns=['CHANNEL'], inplace=True)

database['REAL_PRICE'] = pd.to_numeric(database['REAL_PRICE'], errors='coerce')
database['ORDER_PRICE'] = pd.to_numeric(database['ORDER_PRICE'])
database['COEF'] = database['REAL_PRICE']/database['ORDER_PRICE']
coef = database['COEF'].mean()
database['REAL_PRICE'] = database.apply(lambda row: row['ORDER_PRICE']*coef if pd.isnull(row['REAL_PRICE']) else row['REAL_PRICE'], axis=1)

database['ORDER_DESC'] = database['ORDER_DESC'].str.replace('↑↑', ' ')
database['ORDER_INFO'] = database['ORDER_INFO'].str.replace('\n', ' ')

words = ''.join(database['ORDER_DESC'].astype(str)) + ' '.join(database['ORDER_INFO'].astype(str))
count = Counter(words.split(' ')).most_common(150)
for word in count:
    pattern = re.escape(word[0])
    counts = database['ORDER_DESC'].str.count(pattern) + database['ORDER_INFO'].str.count(pattern)
    database[word[0]] = counts

        

corr_matrix = database.drop(columns=['ORDER_DESC', 'ORDER_INFO']).corr()
least_correlated_features = corr_matrix.abs().mean(axis=1).nsmallest(100)
for feature in least_correlated_features.index:
    if feature in count:
        database.drop(columns=[feature], inplace=True)

database.drop(columns=['ORDER_INFO', 'ORDER_DESC'], inplace=True)

In [41]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from catboost import CatBoostRegressor

In [42]:
X = database.drop(columns=['ORDER_PRICE'])
y = database['ORDER_PRICE']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = DecisionTreeRegressor(random_state=11, criterion='squared_error')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mead = pd.Series(abs(y_test - y_pred)).median()
print(mse, mae, r2, mead)


6.101926775956284 0.7006120218579235 0.9752167703941963 0.1499999999999999


In [43]:
model = CatBoostRegressor(iterations=5000, depth=8, learning_rate=0.005, verbose=False)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mead = pd.Series(abs(y_test - y_pred)).median()
print(mse, mae, r2, mead)

3.045245179169885 0.6786411784780456 0.9876316099402052 0.2904554087887963
